In [1]:
!pip install -q jupyter-dash
!pip install dash_daq
!pip install dash-uploader
#!pip install dash-uploader --pre

     |████████████████████████████████| 9.8 MB 7.4 MB/s 
     |████████████████████████████████| 357 kB 64.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 642 kB 5.1 MB/s 
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669714 sha256=048f15f8074b17e4876a107292b9432018f596f215317d860dfb3e980ea3af08
  Stored in directory: /root/.cache/pip/wheels/fe/54/53/a8d448db5592874db4313240571ca2c069e55f6a6b29bf5847
Successfully built dash-daq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 491 kB 5.2 MB/s 
  Created wheel for dash-uploader: filename=dash_uploader-0.6.0-py3-none-any.whl size=69214 sha256=547172d0a1831c3a63d8d79296d7080b02029642df757e0f4a1eb1335fc7c071
  Stored in directory: /root/.cache/pip/wheels/3b/4d/1c/941d4c28aa60af88391d34dd3f33d03b03372b33551f956b33
Successfully

In [2]:
import os
from google.colab import drive
#drive.mount('/content/gdrive')
#!cp '/content/gdrive/MyDrive/lim_cprd.csv' '/content/'

# imports

In [3]:
import os
import re
import json
import pytz
import numpy as np
import pandas as pd
import seaborn as sns
from string import digits
from google.colab import files
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import datetime
import os
from pprint import pprint
from IPython.core.display import display, HTML
from IPython.display import HTML
import plotly.express as px
from google.colab import data_table
data_table.enable_dataframe_formatter()
import functools
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import random
from scipy import stats
from scipy.stats import skew, mode
from IPython.display import Javascript
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, Button, Layout, GridspecLayout, HBox, VBox, Box, Label
from IPython.display import clear_output
import plotly.graph_objects as go

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.stats import linregress
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import Dash, dcc, html, Input, Output, State, callback_context, dcc
from datetime import date
import dash_daq as daq
#import dash_core_components as dcc


In [5]:
import dash_uploader as du

In [6]:
import base64
import datetime
import io

from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table

import pandas as pd

# Globals

In [7]:
line_x_axis_cat = ['reduced_symbols','tradePlanTemplate','full_symbols','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription', 'costbasis', 'quantity']
line_x_axis = ['closedate','Datewise','reduced_symbols','realizedpnl',' percent_pnl', 'costbasis', 'quantity','tradePlanTemplate','costbasis','quantity','full_symbols','time-in-trade','trade_rate','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription','time_bracket']
line_y_axis = ['realizedpnl', ' percent_pnl', 'time-in-trade', 'trade_rate', 'Count', 'costbasis', 'quantity']
line_colors = ['reduced_symbols','tradePlanTemplate','full_symbols','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription']
axis_options = ['realizedpnl',' percent_pnl','costbasis','quantity','closedate','strategy','strategydescription','tradePlanTemplate','Day of Week','reduced_symbols','full_symbols','time-in-trade','trade_rate','W/L','Long/Short Trades','Datewise','M','time_bracket','setup_type','setupFilters_type','entries_type','exits_types']
ctgr_options = ['W/L','reduced_symbols','full_symbols','Long/Short Trades',' percent_pnl','trade_rate','strategy','strategydescription','tradePlanTemplate','time-in-trade','realizedpnl','Day of Week','M', 'costbasis', 'quantity']
symb_options = ['W/L','reduced_symbols','full_symbols','Long/Short Trades','strategy','strategydescription','tradePlanTemplate']
size_options = ['Count' ,'trade_rate','time-in-trade','realizedpnl',' percent_pnl', 'costbasis', 'quantity']
fcet_options = ['W/L', 'Long/Short Trades']
hver_options = ['strategy','tradePlanTemplate','lifecycle_det','setup_type','setup_det','setupFilters_type','setupFilters_det','options_det','entries_type','entries_det','exits_types','exits_dict','strategydescription','realizedpnl','time-in-trade','trade_rate','full_symbols','reduced_symbols','Day of Week', 'costbasis', 'quantity']

result_cats = ['realizedpnl', ' percent_pnl', 'time-in-trade', 'trade_rate', 'costbasis', 'quantity']

plt.rcParams["figure.figsize"] = [15, 9]

global_json_var = [] # to keep a tab of columns

keyval_key = None

# Lambda Functions

In [8]:
clean_symbols = lambda x: re.search(r"[a-z]*", x, re.IGNORECASE).group()
cgr = lambda x: "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
cnv = lambda x: (x-x%1)*100+(x%1)*60

get_json_dict = lambda x: json.loads(x)
get_json_keys = lambda x: [k for k in get_json_dict(x).keys()]
get_json_keysLen = lambda x: len([k for k in get_json_dict(x).keys()])
get_json_val1 = lambda x: np.nan if get_json_key1(x) is np.nan else get_json_dict(x)[get_json_key1(x)]

converter = lambda x: x.dt.tz_convert(eastern)
rate_calc = lambda x: np.round(x['realizedpnl']/x['time-in-trade'],4)
win_chk = lambda x: 'win' if x>0 else 'loss'
LScnv = lambda x: 'short' if x=='sell_short' else 'long'


# Helper Functions

In [9]:
def get_json_key1(x):
  try:
    return get_json_keys(x)[0]
  except:
    return np.nan


def clean_options(inp):
  try:
    stk_opts = []
    for ckey in inp['optionChoices']['choices'].keys():
      choices = inp['optionChoices']['choices'][ckey]
      stk_opts += [clean_symbols(x) for x in choices]
    stk_opts = list(dict.fromkeys(stk_opts))
    inp['optionChoices'] = stk_opts
    return inp
  except:
    try:
      print(list(inp['optionChoices']['choices'].keys())[0])
    except:
      pass
    return inp

def time_bins(inp, bins=1440):

  time_slots = cnv(np.arange(24/bins, 24+24/bins, 24/bins)).astype(int).tolist()
  #lbls = cnv(np.arange(0, 24, 24/bins)).astype(int).astype(str).tolist()
  lbls = np.arange(0, 24, 24/bins).tolist()

  for i in range(len(time_slots)):
    if inp<time_slots[i]:
      return lbls[i]
  return np.nan

def time_bins_lim(inp, bins=96):

  time_slots = cnv(np.arange(24/bins, 24+24/bins, 24/bins)).astype(int).tolist()
  #lbls = cnv(np.arange(0, 24, 24/bins)).astype(int).astype(str).tolist()
  lbls = np.arange(0, 24, 24/bins).tolist()

  for i in range(len(time_slots)):
    if inp<time_slots[i]:
      return lbls[i]
  return np.nan

def process_stock_list(df, limit_list):
  selected_stocks = []
  if limit_list is () :
    limit_list = ['All']
  for selection in limit_list:
    if selection == 'None':
      pass
    elif selection == 'All':
      selected_stocks += df.reduced_symbols.unique().tolist()
    elif selection == 'Quick Rising':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].max().sort_values().reset_index()['reduced_symbols'].iloc[-4:].tolist()
    elif selection == 'Worst':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].min().sort_values().reset_index()['reduced_symbols'].iloc[:2].tolist()
    elif selection == 'Safest':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].min().sort_values().reset_index()['reduced_symbols'].iloc[-4:].tolist()
    elif selection == 'Most Traded':
      selected_stocks += df['reduced_symbols'].value_counts().reset_index()['index'].iloc[0:6].tolist()
    elif selection == 'Most Impactful':
      _tmp = df.groupby('reduced_symbols')['realizedpnl'].sum().sort_values().reset_index()
      selected_stocks += np.concatenate([ _tmp['reduced_symbols'].iloc[:2].to_numpy(), _tmp['reduced_symbols'].iloc[-2:].to_numpy() ]).tolist()
    else:
      selected_stocks += [selection]
  selected_stocks = list(dict.fromkeys(selected_stocks))
  return selected_stocks

def filter_nans(inp):
  for col in inp.columns:
    inp = inp[inp[col]!=np.nan]
    inp = inp[inp[col]!=None]
    inp = inp.dropna()
  return inp

def filter_outliers(inp, columns_to_check = ['realizedpnl']):
  for col in columns_to_check:
    inp = inp[(np.abs(stats.zscore(inp[col])) < 3)]
  return inp



# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  #print("Enter Resize")
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
  #print("Resized")
#get_ipython().events.register('pre_run_cell', resize_colab_cell)


def get_var():
  global global_json_var
  return global_json_var
def set_var():
  global global_json_var
  global_json_var = []

def extract_roots(inp):
  try:
    inp = json.loads(inp)
  except:
    print(inp)
    return ''
  roots = ''
  for key in inp.keys():
    roots += str(key)
    roots += ', '
  return roots[:-2]

def extract_leaves_list(inp, root=None):
  outs = []
  if isinstance(inp, dict):
    for key in inp.keys():
      if isinstance(inp[key], dict):
        if root is None:
          outs += extract_leaves_list(inp[key], key)
        else:
          outs += extract_leaves_list(inp[key], root)
      else:
        if (root is None) or (root is ''):
          outs += [ str(key)+' == '+str(inp[key]) ]
        else:
          try:
            if key[4]=='-':
              outs += [ str(root)+' :: '+str(root)+' == '+str(inp[key]) ]
            else:
              outs += [ str(root)+' :: '+str(key)+' == '+str(inp[key]) ]
          except:
            outs += [ str(root)+' :: '+str(key)+' == '+str(inp[key]) ]
    return outs
  else:
    return []

def extract_leaves(inp, root=None, filters=[]):
  global global_json_var
  try:
    inp = json.loads(inp)
  except:
    print(inp)
    return {}
  inp = extract_leaves_list(inp, root)
  for filt in filters:
    _c = inp.copy()
    for item in _c:
      if filt in item:
        inp.remove(item)
  out_dict = {}
  for entry in inp:
    entry = entry.split(' == ')
    if entry[0] not in global_json_var:
      global_json_var += [entry[0]]
    try:
      out_dict[entry[0]] = float(entry[1])
    except:
      out_dict[entry[0]] = str(entry[1])
  #print(out_dict)
  return out_dict
  #return inp
  #return '\n'.join(inp)


def set_keyval(key):
  global keyval_key
  keyval_key = key
def keyval_finderr(inp):
  try:
    return inp[keyval_key]
  except:
    return np.nan


def expand_into_columns(df_full, inp_col, col_list):
  for col in col_list:
    #keyval_finder = lambda x: (x[col] if col in x.keys() else np.nan) if isinstance(x, dict) else np.nan
    keyval_finder = lambda x: (x[col] if col in x.keys() else -1) if isinstance(x, dict) else -1
    df_full[col] = df_full[inp_col].apply(keyval_finder)
  return df_full.copy()

# Plot Functions

In [10]:
def time_dist_plots(wdf):
  stkslcts = wdf.reduced_symbols.unique()
  ax = wdf[wdf['reduced_symbols']==stkslcts[0]].sort_values('time_bracket').plot.scatter(x="time_bracket", y="realizedpnl",rot=45, color = cgr(None), label=stkslcts[0], alpha=0.7)
  plt.locator_params(axis='x', nbins=6)
  for i in range(1,len(stkslcts)):
    wdf[wdf['reduced_symbols']==stkslcts[i]].sort_values('time_bracket').plot.scatter(x="time_bracket", y="realizedpnl",rot=45, color = cgr(None), label=stkslcts[i], ax=ax, alpha=0.7)
  #plt.grid(b=True, which='major')
  #ax.set_yscale('symlog', linthreshy=600)
  if np.abs(wdf['realizedpnl'].max())>400:
    plt.ylim(-300,300)
  plt.title('Trades Distribution w.r.t. Time Brackets (UTC)')

  plt.show()
  """
  ax = wdf[['time_bracket']].value_counts().sort_index().plot.bar(stacked=True, rot=45, color = cgr(None), label=stkslcts[0], alpha=1)
  plt.grid(b=True, which='major')
  plt.show()
  """
  wdf[['time_bracket_lim','reduced_symbols','Count']].groupby(['reduced_symbols','time_bracket_lim']).sum().reset_index().pivot(index='time_bracket_lim', columns='reduced_symbols', values='Count').replace({np.nan:0}).plot.bar(stacked=True, rot=45)
  plt.grid(b=True, which='major')
  plt.title("Number of trades made during each time bracket")
  #plt.locator_params(axis='x', nbins=6)
  plt.show()

def cumulative_plots(DF):
  wdf = DF.copy()
  stkslcts = wdf.reduced_symbols.unique()
  for stock in stkslcts:
    cum_stocks = lambda x: x['realizedpnl'] if x['reduced_symbols']==stock else 0
    r_stocks = lambda x: x['trade_rate'] if x['reduced_symbols']==stock else 0
    pos_vals = lambda x: x['c-'+stock] if x['c-'+stock]>0 else 0
    neg_vals = lambda x: x['c-'+stock] if x['c-'+stock]<0 else 0
    wdf[stock] = wdf[['realizedpnl', 'reduced_symbols']].apply(cum_stocks, axis=1)
    wdf['c-'+stock] = wdf[stock].cumsum()
    wdf['r-'+stock] = wdf[['trade_rate', 'reduced_symbols']].apply(r_stocks, axis=1)
    wdf['pos-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(pos_vals, axis=1)
    wdf['neg-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(neg_vals, axis=1)
  _tmp = ['pos-'+x for x in stkslcts]
  wdf['pos_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf['neg_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf.set_index('closedate', inplace=True)

  _tmp = ['pnl_accumulated']+['c-'+x for x in stkslcts]
  _rtmp = ['r-'+x for x in stkslcts]
  ax = wdf[_tmp].plot(kind="line", rot=45)
  plt.grid(b=True, which='major')
  ax.set_yscale('symlog', linthreshy=600)
  plt.title('Cumulative PNL of Trades (Total and symbol-wise)')
  plt.show()

  #wdf[['reduced_symbols','realizedpnl']].reset_index().pivot_table(index = 'closedate',columns='reduced_symbols',values='realizedpnl')
  #_tmp = wdf[['reduced_symbols','realizedpnl','Datewise']].groupby(['Datewise','reduced_symbols']).sum().reset_index().pivot(index='Datewise', columns='reduced_symbols', values='realizedpnl').replace({np.nan:0})
  _tmp = wdf[['reduced_symbols','realizedpnl']].reset_index().groupby(['closedate','reduced_symbols']).sum().reset_index().pivot(index='closedate', columns='reduced_symbols', values='realizedpnl').replace({np.nan:0})
  for col in _tmp.columns:
    _tmp[col] = _tmp[col].cumsum()
  #ax = _tmp.plot.bar(stacked=True, rot=45)
  ax = _tmp.plot.area(stacked=False, rot=45)
  wdf['realizedpnl'].cumsum().plot.line( linestyle = '-', linewidth = '2', style=['b--'], ax=ax)
  plt.locator_params(axis='x', nbins=6)
  plt.tick_params(
      axis='x',          # changes apply to the x-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,         # ticks along the top edge are off
      labelbottom=False) # labels along the bottom edge are off
  #ax.set_yscale('symlog', linthreshy=600)
  plt.grid()
  plt.title('Cumulative PNL of Trades (Total and symbol-wise)')
  plt.show()

def PNL_Spread(wdf):
  ax = wdf.plot.scatter(x="reduced_symbols", y="realizedpnl")
  plt.grid(b=True, which='major')
  ax.set_yscale('symlog', linthreshy=100)
  plt.title('Realized_pnl Spread vs Symbols')
  plt.show()

def returns_indicator(wdf):
  stat = wdf[['strategy','reduced_symbols','realizedpnl']].groupby(['strategy','reduced_symbols']).sum()
  stat['color'] = stat['realizedpnl']
  stat['size'] = np.sqrt(np.abs(stat['realizedpnl']))
  while (stat['size'].max()<700):
    stat['size'] = np.power(stat['size'],1.2)
  stat['color'][stat['realizedpnl']>0] = 'Green'
  stat['color'][stat['realizedpnl']<=0] = 'Red'
  stat.reset_index(inplace=True)
  stat.plot.scatter(x="reduced_symbols", y="strategy", c='color', s='size')
  plt.title("Strategy-wise returns indicator")
  plt.show()
  #stat

  stat = wdf[['side','reduced_symbols','realizedpnl']].groupby(['side','reduced_symbols']).sum()
  stat['color'] = stat['realizedpnl']
  stat['size'] = np.sqrt(np.abs(stat['realizedpnl']))
  while (stat['size'].max()<700):
    stat['size'] = np.power(stat['size'],1.2)
  stat['color'][stat['realizedpnl']>0] = 'Green'
  stat['color'][stat['realizedpnl']<=0] = 'Red'
  stat.reset_index(inplace=True)
  stat.plot.scatter(x="reduced_symbols", y="side", c='color', s='size')
  plt.title("Side-wise returns indicator")
  plt.show()

def make_area_plots (DF, pos_neg=True):
  wdf = DF.copy()
  stkslcts = wdf.reduced_symbols.unique()
  for stock in stkslcts:
    cum_stocks = lambda x: x['realizedpnl'] if x['reduced_symbols']==stock else 0
    r_stocks = lambda x: x['trade_rate'] if x['reduced_symbols']==stock else 0
    pos_vals = lambda x: x['c-'+stock] if x['c-'+stock]>0 else 0
    neg_vals = lambda x: x['c-'+stock] if x['c-'+stock]<0 else 0
    wdf[stock] = wdf[['realizedpnl', 'reduced_symbols']].apply(cum_stocks, axis=1)
    wdf['c-'+stock] = wdf[stock].cumsum()
    wdf['r-'+stock] = wdf[['trade_rate', 'reduced_symbols']].apply(r_stocks, axis=1)
    wdf['pos-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(pos_vals, axis=1)
    wdf['neg-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(neg_vals, axis=1)
  _tmp = ['pos-'+x for x in stkslcts]
  wdf['pos_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf['neg_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf.set_index('closedate', inplace=True)

  data = wdf[_tmp][wdf[_tmp].sum(axis=1)<0]
  #plt.locator_params(axis='x', nbins=10)
  #ax = data.plot.bar(stacked=True)
  data_perc = data.divide(data.sum(axis=1), axis=0)
  ax = data_perc.plot.area(stacked=True, alpha=0.7)
  wdf[['neg_acm']].plot(kind='line', secondary_y=True, linestyle = '-', linewidth = '2', style=['r--'], ax=ax)
  plt.title('Negative Area Plot')
  plt.show()

  _tmp = ['pos-'+x for x in stkslcts]
  data = wdf[_tmp][wdf[_tmp].sum(axis=1)>0]
  data_perc = data.divide(data.sum(axis=1), axis=0)*-1
  ax = data_perc.plot.area(stacked=True)
  wdf[['pos_acm']].plot(kind='line', secondary_y=True, linestyle = '-', linewidth = '2', style=['b--'], ax=ax)
  plt.title('Positive Area Plot')
  plt.show()




# Interactive Plot Functions

In [11]:
def make_parallel(df_stkLim, rdims):
  try:
    #cols = [color_col]+list(dims)
    cols = list(rdims)
    filt = df_stkLim[list(filter(None, cols))].copy()
    #filt = filter_nans(self.df_stkLim[list(filter(None, cols))].copy())
    #fig = px.parallel_coordinates(filt, color=color_col, dimensions=list(dims), width=1600, height=800)
    fig = px.parallel_coordinates(filt, dimensions=list(rdims))#, width=1600, height=800)
    fig.update_layout(paper_bgcolor="LightSteelBlue")
    return fig
    display(HTML(fig.to_html()))
  except Exception as E:
    raise E
    print("Couldn't make graph due to following error: "), print(E)

def make_scatter(df_stkLim, xaxis,yaxis=None,size_col=None,color_col=None, symbol_col=None, facet_col=None, hover_col=None):
  
  try:
    
    cols = [xaxis, yaxis, size_col, color_col, symbol_col, facet_col, hover_col]
    col2 = [xaxis, yaxis, color_col, symbol_col, facet_col, hover_col]
    #try:
    if (size_col in col2) and (size_col is not None):
      filt = filter_nans(df_stkLim[list(filter(None, col2))].copy())
      filt[size_col+'_'] = filt[size_col] 
      size_col = size_col+'_'
    else:
      #pass
      #filt = df_stkLim
      filt = filter_nans(df_stkLim[list(filter(None, cols))].copy())
    #filt = filt.groupby([xaxis,yaxis]).agg({size_col:'sum', color_col:'min', symbol_col:'min', facet_col:'min', hover_col:'min'})
    
    agg_cols = [size_col, color_col, symbol_col, facet_col, hover_col]
    agg_mets = ['sum', 'first', 'first', 'first', 'first']
    agg_dict={}
    
    for i in range(len(agg_mets)):
      if agg_cols[i] is not None:
        agg_dict[agg_cols[i]] = agg_mets[i]
    if agg_dict != {}:
      filt = filt.groupby([xaxis,yaxis]).agg(agg_dict)
    try:
      filt[size_col] = np.abs(filt[size_col])
      filt.rename(columns={size_col: 'ABS('+size_col+')'}, inplace=True)
      size_col = 'ABS('+size_col+')'
    except:
      size_col = None
    filt = filter_nans(filt)
    fig = px.scatter(filt.reset_index(), x=xaxis, y=yaxis, color=color_col, size=size_col, symbol=symbol_col, facet_col=facet_col, hover_data=[hover_col])#, width=1600, height=800)
    fig.update_layout(paper_bgcolor="LightSteelBlue")
    return fig
    #display(HTML(fig.to_html()))
    ##resize_colab_cell()
    display(HTML(fig.to_html()))
  except Exception as E:
    raise E
    print("Couldn't make graph due to following error: "), print(E)


def make_3dscatter(df_stkLim, xaxis, yaxis, zaxis, size_col=None, color_col=None, symbol_col=None, hover_col=None):
  try:
    cols = [xaxis, yaxis, zaxis, size_col, color_col, symbol_col, hover_col]
    col2 = [xaxis, yaxis, zaxis, color_col, symbol_col, hover_col]
    #try:
    if (size_col in col2) and (size_col is not None):
      filt = filter_nans(df_stkLim[list(filter(None, col2))].copy())
      filt[size_col+'_'] = filt[size_col] 
      size_col = size_col+'_'
    else:
      filt = filter_nans(df_stkLim[list(filter(None, cols))].copy())
    #filt = filt.groupby([xaxis,yaxis]).agg({size_col:'sum', color_col:'min', symbol_col:'min', hover_col:'min'})
    agg_cols = [size_col, color_col, symbol_col, hover_col]
    agg_mets = ['sum', 'first', 'first', 'first']
    agg_dict={}
    for i in range(len(agg_mets)):
      if agg_cols[i] is not None:
        agg_dict[agg_cols[i]] = agg_mets[i]
    if agg_dict != {}:
      filt = filt.groupby([xaxis,yaxis,zaxis]).agg(agg_dict)
    try:
      filt[size_col] = np.abs(filt[size_col])
      filt.rename(columns={size_col: 'ABS('+size_col+')'}, inplace=True)
      size_col = 'ABS('+size_col+')'
    except:
      size_col = None
    filt = filter_nans(filt)
    fig = px.scatter_3d(filt.reset_index(), x=xaxis, y=yaxis, z=zaxis, color=color_col, size=size_col, symbol=symbol_col, hover_data=[hover_col])#, width=1600, height=800)
    fig.update_layout(paper_bgcolor="LightSteelBlue")
    return fig
    #display(HTML(fig.to_html()))
    ##resize_colab_cell()
    display(HTML(fig.to_html()))
  except Exception as E:
    raise E
    print("Couldn't make graph due to following error: "), print(E)

def make_line(df_stkLim, xaxis, yaxis, color_col=None):
  try:
    cols = [xaxis, yaxis, color_col]
    filt = filter_nans(df_stkLim[list(filter(None, cols))].copy())
    filt = filter_nans(filt)
    if color_col:
      filt = filt.groupby([xaxis,color_col]).agg('sum')
    else:
      filt = filt.groupby([xaxis]).agg('sum')
    filt = filter_nans(filt)
    filt.reset_index(inplace=True)
    filt = filt.sort_values(by=xaxis)
    if color_col:
      filt = filt.sort_values(by=color_col)
    filt = filt.sort_values(xaxis).reset_index(drop=True)
    if xaxis in line_x_axis_cat:
      fig = px.scatter(filt, x=xaxis, y=yaxis, color=color_col)#, width=1600, height=800)
    else:
      fig = px.line(filt, x=xaxis, y=yaxis, color=color_col)#, width=1600, height=800)
    fig.update_layout(paper_bgcolor="LightSteelBlue")
    return fig
    ##resize_colab_cell()
    display(HTML(fig.to_html()))
    
  except Exception as E:
    raise E
    print("Couldn't make graph due to following error: "), print(E)
def make_treemap_anim(df_stkLim, out=None):
  def make_treemap_params(row):
    pos_sum = 0
    neg_sum = 0
    try:
      period_string = 'Year:'+row[list(row.keys())[0]][:4]+' Month:'+row[list(row.keys())[0]][4:6]+' Week:'+row[list(row.keys())[0]][6:]
    except:
      period_string = row[list(row.keys())[0]]
    param_dict = {'parents':[], 'labels':[],'values':[], 'Period':[period_string]}
    for key in row['realizedpnl'].keys():
      param_dict['labels'] += [key]
      if row['realizedpnl'][key] >= 0:
        param_dict['parents'] += ['gains']
        param_dict['values'] += [row['realizedpnl'][key]]
        pos_sum += row['realizedpnl'][key]
      else:
        param_dict['parents'] += ['losses']
        param_dict['values'] += [row['realizedpnl'][key] * (-1)]
        neg_sum += (-1)*row['realizedpnl'][key]
    param_dict['labels'] = ['gains', 'losses'] + param_dict['labels']
    param_dict['parents'] = [param_dict['Period'],param_dict['Period']] + param_dict['parents']
    param_dict['values'] = [pos_sum,neg_sum] + param_dict['values']
    return param_dict


  period = 'YMW'
  tmp_data = df_stkLim[[period, 'reduced_symbols', 'realizedpnl']].copy()
  tmp_data = tmp_data.groupby([period,'reduced_symbols']).agg('sum').unstack(level=-1).fillna(0).cumsum(axis=0).reset_index()
  tmp_data ['TM_params'] = tmp_data.apply(make_treemap_params, axis=1)
  tmp_data

  proc_data = tmp_data['TM_params']
  proc_frames = [go.Frame(data=[go.Treemap(labels = proc_data[i]['labels'], parents = proc_data[i]['parents'], values =  proc_data[i]['values'],root_color="lightgrey",branchvalues = "total"
  ,textinfo = "label+value+percent parent+percent entry+percent root"  )])
                  for i in range(1,len(proc_data))]
  button_styles = {"buttons": 
                    [ {"args": 
                      [None, {"frame": {"duration": 500, "redraw": False},
                              "fromcurrent": True,
                              "transition": {"duration": 300, "easing": "quadratic-in-out"}}],
                      "label": "Play",
                      "method": "animate" },
                    {"args":
                      [[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0}}],
                      "label": "Pause",
                      "method": "animate"}],
                    "direction": "left",
                    "pad": {"r": 10, "t": 87},
                    "showactive": False,
                    "type": "buttons",
                    "x": 0.11,
                    "xanchor": "right",
                    "y": 0.0,
                    "yanchor": "top"}
  fig = go.Figure(
    data=[
      go.Treemap(marker_colors = [px.colors.qualitative.Plotly[0], px.colors.qualitative.Plotly[1]],
        labels = proc_data[0]['labels'],
        parents = proc_data[0]['parents'],
        values =  proc_data[0]['values'],
        #textinfo = "label+value+percent parent+percent entry+percent root",
        root_color="lightgrey")
    ],
    layout=
      go.Layout(
          #width=1200, height=800,
          #title="Start Title",
          updatemenus=[button_styles]
      )
    ,
    frames=proc_frames
  )

  fig.update_layout(treemapcolorway = [px.colors.qualitative.Plotly[0], px.colors.qualitative.Plotly[1]], margin = dict(t=50, l=25, r=25, b=25))
  fig.update_layout(paper_bgcolor="LightSteelBlue")
  return fig
  with out:
    display(HTML(fig.to_html()))
    #fig.show()
  

# Interactive Graph UI Functions

In [12]:
  def make_initial_UI(self, EH_function, nested = None):
    resize_colab_cell()
    selector_options_s = ['All', 'Most Traded', 'Most Impactful', 'Safest', 'Worst', 'Quick Rising'] + self.df_full.reduced_symbols.unique().tolist()
    selector_options = ['Cumulative Plots', 'Time Distribution Plots', 'PNL Spread vs Symbols', 'Returns Indicator Plots', 'Profit/Loss Area Plots', "Interactive Scatter Plot", "Interactive 3D Scatter Plot", "Interactive Aggregate Plot", "Interactive Parallel Plot", "Treemap Animation"]
    # Defining Widgets
    radbt = widgets.RadioButtons(options=['All Data', 'Only Live', 'Only Backtest'], value='All Data', description='Select Data', disabled=False)
    date1 = widgets.DatePicker(value=self.df_full['E_Dates'].iloc[0])
    date2 = widgets.DatePicker(value=self.df_full['E_Dates'].iloc[-1])
    slctr = widgets.SelectMultiple(options=selector_options_s, description='Stocks:', disabled=False, value = (selector_options_s[0],) )
    chkbx = widgets.Checkbox( value=False, description='Filter outlier trade entries?', disabled=False)
    embbt = widgets.ToggleButtons( options=selector_options, description='Plot Selection:', disabled=False, button_style='', tooltips=selector_options, value=selector_options[5] )

    # Combine the widgets into a single display layout
    form_item_layout = Layout(display='flex', justify_content='space-between')
    dates = HBox([Label(value='',layout=Layout(width='60px')),Label(value='Date Selection:'),VBox([date1,date2])])
    data_opts = VBox([radbt,chkbx], layout=form_item_layout)
    row2 = HBox([data_opts, slctr, dates])
    grid = VBox([embbt, row2], layout=form_item_layout)

    # Define one single output widget
    output = widgets.Output()

    # Define only one event handler that will be called for each widget
    def internal_event_handler(change):
      # The called function must have knowledge of which inputs to expect
      output.clear_output()
      if self.cur_output is not None:
        clear_output()
        display(grid)
        display(output)
      EH_function(output, [radbt.value, date1.value, date2.value, slctr.value, chkbx.value, embbt.value])
    
    # Attach the event handler to all widgets using their respective .observe methods
    radbt.observe(internal_event_handler, names='value')
    date1.observe(internal_event_handler, names='value')
    date2.observe(internal_event_handler, names='value')
    slctr.observe(internal_event_handler, names='value')
    chkbx.observe(internal_event_handler, names='value')
    embbt.observe(internal_event_handler, names='value')

    # Display the output and layouts
    # If nested, use the display from previous widget
    if nested is not None:
      with nested:
        display(grid)
        display(output)
        with output:
          # initialize the internal output
          print("")
    else:
      display(grid)
      display(output)
      with output:
        # initialize the internal output
        print("")
    # Call internal function to initiliaze proper state
    internal_event_handler(None)

    # return both display items for further use
    return grid, output


# Preproc Functions

In [13]:
def upload_file(filename=''):
  if os.path.exists(filename):
    df_inp = pd.read_csv(filename)
  else:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df_inp = pd.read_csv(filename)
  return df_inp


def preprocess_df(df_inp, trades_filter=10):
  df_full = pd.DataFrame()
  df_liveLim = pd.DataFrame()
  df_dateLim = pd.DataFrame()

  dates_range = []
  selector = None

  persistent_plot_params=[]
  persistent_chk = ''

  json_column_keys_dict = {}
  json_options = []
  global global_json_var

  cur_output = None

  debug = None
  # Convert date columns to pandas datetime format and EST
  date_cols = ['entrydate', 'closedate']
  eastern = pytz.timezone('US/Eastern')
  df_inp[date_cols[0]], df_inp[date_cols[1]] = pd.to_datetime(df_inp[date_cols[0]], utc=True).dt.tz_convert(eastern), pd.to_datetime(df_inp[date_cols[1]], utc=True).dt.tz_convert(eastern)

  # Make an int column to use as reference index 
  df_inp['idxdate'] = df_inp['entrydate'].dt.strftime("%y%m%d%H%M%S").astype(int)
  df_inp.sort_values('idxdate', inplace=True)
  df_inp.set_index('idxdate', inplace=True)

  ss_i = 0
  # Clean the symbol column
  df_inp['reduced_symbols'] = df_inp['symbol'].apply(clean_symbols)

  # Replace less-traded stocks as 'OTHER'
  df_inp['full_symbols'] = df_inp['reduced_symbols']  # Keep a copy of the column
  _tmp = df_inp['reduced_symbols'].value_counts().reset_index()
  other_stocks = _tmp[_tmp['reduced_symbols']<trades_filter]['index'].unique().tolist()
  for stk in other_stocks:
    df_inp['reduced_symbols'].replace({stk: 'OTHER'}, inplace=True)
  
  # Add time-in-trade column and handle faulty rows, replacing Faulty Values with default (1) or removing the respective rows
  df_inp['time-in-trade'] = (df_inp['closedate'] - df_inp['entrydate']).astype('timedelta64[s]')
  df_inp = df_inp[df_inp['time-in-trade'] > 0]
  #df_inp['time-in-trade'][ df_inp['time-in-trade']<=0 ] = 1

  # Make a working copy, and delete extra columns in inp to make total<=20 (for colab display)
  df_full = df_inp.copy()
  df_inp.drop('symbol', inplace=True, axis=1)
  df_inp.drop('full_symbols', inplace=True, axis=1)

  # Add a rates and win/loss columns
  df_full['trade_rate'] = df_full[['realizedpnl', 'time-in-trade']].apply(rate_calc, axis=1)
  df_full['W/L'] = df_full['realizedpnl'].apply(win_chk)

  # Convert to long/short sides
  df_full['Long/Short Trades'] = df_full['side'].apply(LScnv)

  
  # Add additional columns for later grouping and filtering operations
  df_full['E_Dates'] = df_full['entrydate'].dt.date#.apply(converter)
  df_full['Datewise'] = df_full['closedate'].dt.date#.apply(converter)
  df_full['C_Time'] = df_full['closedate'].dt.time
  df_full['H'] = df_full['entrydate'].dt.strftime("%H").astype(int)
  df_full['D'] = df_full['entrydate'].dt.strftime("%d").astype(int)
  df_full['M'] = df_full['entrydate'].dt.strftime("%m").astype(int)
  df_full['hm'] = df_full['entrydate'].dt.strftime("%H%M"[:4]).astype(int)
  df_full['hms'] = df_full['entrydate'].dt.strftime("%H:%M:%S")
  df_full['YM'] = df_full['entrydate'].dt.strftime("%Y%m")
  df_full['YMW'] = df_full['entrydate'].dt.strftime("%Y%m%W")
  df_full['Day of Week'] = df_full['entrydate'].dt.day_name()
  df_full['time_bracket'] = df_full['hm'].apply(time_bins)
  df_full['time_bracket_lim'] = df_full['hm'].apply(time_bins_lim)
  df_full['Count'] = 1

  # Translate JSON columns to respective details
  
  set_var()
  df_full['lifecycle_det'] = df_full['lifecycle'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
  json_column_keys_dict['lifecycle'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'lifecycle_det', json_column_keys_dict['lifecycle'])
  

  set_var()
  df_full['setup_type'] = df_full['setup'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
  df_full['setup_det'] = df_full['setup'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x, filters=['pattern']))
  json_column_keys_dict['setup'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'setup_det', json_column_keys_dict['setup'])

  set_var()
  df_full['setupFilters_type'] = df_full['setupFilters'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
  df_full['setupFilters_det'] = df_full['setupFilters'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
  json_column_keys_dict['setupFilters'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'setupFilters_det', json_column_keys_dict['setupFilters'])

  set_var()
  df_full['options_det'] = df_full['options'].apply(lambda x: np.nan if x is np.nan else clean_options(extract_leaves(x)))
  json_column_keys_dict['options'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'options_det', json_column_keys_dict['options'])

  set_var()
  df_full['entries_type'] = df_full['entries'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
  df_full['entries_det'] = df_full['entries'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
  json_column_keys_dict['entries'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'entries_det', json_column_keys_dict['entries'])

  set_var()
  df_full['exits_types'] = df_full['exits'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
  df_full['exits_dict'] = df_full['exits'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x, filters=['paused', 'tacticId']))
  json_column_keys_dict['exits'] = get_var().copy()
  df_full = expand_into_columns(df_full, 'exits_dict', json_column_keys_dict['exits'])

  for key in json_column_keys_dict.keys():
    json_options += json_column_keys_dict[key]
  
  # Set the variable defining possible range of date selection
  dates_range = df_full.E_Dates.unique()


  return df_full, df_inp, dates_range, json_column_keys_dict, json_options

# WDF Functions

In [14]:
def make_wdf(df_base, opts):
  #opts = json.loads(opts)
  start, end, Liv, Bck, Flt = opts
  start = datetime.datetime.strptime(str(start), '%Y-%m-%d').date()
  end = datetime.datetime.strptime(str(end), '%Y-%m-%d').date()
  if Liv:
    if Bck:
      bckTest='All Data'
    else:
      bckTest='Only Live'
  else:
    if Bck:
      bckTest='Only Bck'
    else:
      bckTest='All Data'
  if bckTest == 'All Data':
    wdf = df_base.copy()
  elif bckTest == 'Only Live':
    wdf = df_base[df_base['accountId']!='BACKTEST'].copy()
  elif bckTest == 'Only Bck':
    wdf = df_base[df_base['accountId']=='BACKTEST'].copy()
  else:
    wdf = df_base.copy()
  
  try:
    if (start>wdf['E_Dates'].iloc[-1]) or (end<wdf['E_Dates'].iloc[0]) or (start>end):
      pass
  except:
    wdf.E_Dates = pd.to_datetime(wdf.E_Dates, format='%Y-%m-%d').dt.date
  if (start>wdf['E_Dates'].iloc[-1]) or (end<wdf['E_Dates'].iloc[0]) or (start>end):
    pass
  else:
    # find start/end indices
    i_start, i_end = np.where(wdf.E_Dates >= start)[0][0], np.where(wdf.E_Dates <= end)[0][-1]
    
    if (i_start >= i_end):
      pass
    else:
      
      # apply indices
      wdf = wdf.iloc[i_start:i_end].copy()
      
      # Recalculate cumulative pnl
      wdf['pnl_accumulated'] = wdf.realizedpnl.cumsum()
      #print("\rStart Date: {}\t\tEnd Date: {}".format(i_start,i_end), end="\n")
  if Flt:
    wdf = filter_outliers(wdf.copy(), columns_to_check = ['realizedpnl', 'time-in-trade', 'trade_rate'])
    #print('Filtered outliers in {}'.format(['realizedpnl', 'time-in-trade', 'trade_rate']))
  #print(limit_list)
  
  wdf['pnl_accumulated'] = wdf.realizedpnl.cumsum()
  
  return wdf

# Define Dataframes

In [15]:
#df_inpp = upload_file('lim_cprd.csv')
#json_txt = df_inpp.iloc[list(range(19,31)),:].to_json(date_format='iso', orient='split')
#df_inp = pd.read_json(json_txt, orient='split')
#df_inp
#df_inp = upload_file('lim_cprd.csv')
df_inp_txt = '{"columns":["symbol","side","realizedpnl"," percent_pnl","costbasis","quantity","entrydate","closedate","strategy","strategydescription","tradePlanTemplate","accountId","day","lifecycle","setup","setupFilters","options","entries","exits"],"index":[19,20,21,22,23,24,25,26,27,28,29,30],"data":[["GME","buy",-1.22,-0.086043862,1417.8815,7,"2021-06-25T11:14:05.000-0400","2021-06-25T11:14:07.000-0400","AD_HOC","AD_HOC","Sim on Live data","BACKTEST","25\/06\/2021",null,null,null,null,null,null],["GME","sell_short",-6.67,-0.133949192,4979.5,23,"2021-06-24T12:30:07.000-0400","2021-06-24T12:30:10.000-0400","CANDLE_PATTERN","INSIDE_CANDLE-BREAK_HIGH-BREAK_LOW","1-2-2 S sim","BACKTEST","24\/06\/2021",null,null,null,null,null,null],["SCR","sell_short",-8.16,-0.032744415,24920.28,1023,"2021-06-23T09:38:23.000-0400","2021-06-23T09:39:11.000-0400","ORB",null,null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["AMC","buy",312.72,1.252923769,24959.22,429,"2021-06-23T09:35:37.000-0400","2021-06-23T09:38:43.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["ABCL","sell_short",26.95,0.108213983,24904.36,1222,"2021-06-23T09:36:50.000-0400","2021-06-23T09:38:39.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["TSLA","buy",132.81,0.532186974,24955.515,39,"2021-06-23T09:35:07.000-0400","2021-06-23T09:38:02.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["GME","sell_short",-138.51,-0.556815838,24875.37,114,"2021-06-23T09:37:12.000-0400","2021-06-23T09:37:55.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["CLOV","sell_short",-47.58,-0.190755686,24942.9,1830,"2021-06-23T09:37:11.000-0400","2021-06-23T09:37:26.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["BA","sell_short",-63.24,-0.25383828,24913.5,102,"2021-06-23T09:35:01.000-0400","2021-06-23T09:35:45.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["LOOP","buy",213.78,0.852249623,25084.2,1724,"2021-06-23T09:35:04.000-0400","2021-06-23T09:35:36.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["ENTX","sell_short",-109.95,-0.444444444,24738.75,3665,"2021-06-23T09:35:19.000-0400","2021-06-23T09:35:20.000-0400","ORB","ORB",null,"BACKTEST","23\/06\/2021",null,null,null,null,null,null],["MRNA","buy",-26.55,-1.856968001,1429.75,7,"2021-06-18T09:41:33.000-0400","2021-06-18T10:46:29.000-0400","CANDLE_TRIGGER","CANDLE_TRIGGER",null,"BACKTEST","18\/06\/2021",null,null,null,null,null,null]]}'
df_inp = pd.read_json(df_inp_txt, orient = 'split')
df_full, df_inp, dates_range, json_column_keys_dict, json_options = preprocess_df(df_inp)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [16]:
para = make_parallel(df_full, result_cats)
sctr = make_scatter(df_full, 'realizedpnl', ' percent_pnl')
sctr3d = make_3dscatter(df_full, 'realizedpnl', ' percent_pnl', 'quantity')
aggr = make_line(df_full, 'realizedpnl', ' percent_pnl')
tmap = make_treemap_anim(df_full)

In [17]:
st_date = df_full['E_Dates'].iloc[0]
en_date = df_full['E_Dates'].iloc[-1]
df_full = df_full.iloc[:100,:]

# Make Interface

In [18]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


bg_color = '#b0c4de'
fig = go.Figure()

#style_main = { 'height': '800px', 'width': '1600px', 'background-color':bg_color, 'margin': '0'}
style_main = { 'height': '800px', 'width': '1600px', 'background-color':bg_color, 'margin': '0', 'display':'block'}
style_left_pane = { 'height': '100%', 'width': '22%', 'background-color':bg_color, 'float': 'left',  'margin-left':'2%'}
style_right_pane = { 'height': '99%', 'width': '72%', 'background-color':bg_color, 'float': 'right', 'margin-right':'2%' }
style_plot_options = { 'height': '50%', 'width': '100%', 'background-color':bg_color, 'margin': '0', 'float':'bottom' , 'overflow': 'hidden', 'position': 'relative'}
style_graph = { 'height': '50%', 'width': '100%', 'background-color':bg_color, 'margin': '0', 'float':'top' }

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
du.configure_upload(app, r"./content/uploads/")#, use_upload_id=True)
scatter_UI = html.Div([
  html.Div(['xaxis'     , dcc.Dropdown(axis_options, axis_options[0], id='sui_xaxis'     , clearable = False, searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['yaxis'     , dcc.Dropdown(axis_options, axis_options[1], id='sui_yaxis'     , clearable = False, searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['size_col'  , dcc.Dropdown(size_options, id='sui_size_col'  , clearable = True , searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['color_col' , dcc.Dropdown(ctgr_options, id='sui_color_col' , clearable = True , searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['symbol_col', dcc.Dropdown(symb_options, id='sui_symbol_col', clearable = True , searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['facet_col' , dcc.Dropdown(fcet_options, id='sui_facet_col' , clearable = True , searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['hover_col' , dcc.Dropdown(hver_options, id='sui_hover_col' , clearable = True , searchable=True, className='sui')], style = {'width':'300px', 'float': 'left'})
], id='sui_div', style={'display':'none'})

scatter3d_UI = html.Div([
  html.Div(['xaxis'     , dcc.Dropdown(axis_options, axis_options[0], id='sui3d_xaxis'     , clearable = False, searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['yaxis'     , dcc.Dropdown(axis_options, axis_options[1], id='sui3d_yaxis'     , clearable = False, searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['zaxis'     , dcc.Dropdown(axis_options, axis_options[2], id='sui3d_zaxis'     , clearable = False, searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['size_col'  , dcc.Dropdown(size_options, id='sui3d_size_col'  , clearable = True , searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['color_col' , dcc.Dropdown(ctgr_options, id='sui3d_color_col' , clearable = True , searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['symbol_col', dcc.Dropdown(symb_options, id='sui3d_symbol_col', clearable = True , searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['hover_col' , dcc.Dropdown(hver_options, id='sui3d_hover_col' , clearable = True , searchable=True, className='sui3d')], style = {'width':'300px', 'float': 'left'})
], id='3dui_div', style={'display':'none'})

aggr_UI = html.Div([
  html.Div(['xaxis'     ,dcc.Dropdown(line_x_axis, line_x_axis[0], id='aggr_xaxis'     , clearable = False, searchable=True, className='aggr')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['yaxis'     ,dcc.Dropdown(line_y_axis, line_y_axis[1], id='aggr_yaxis'     , clearable = False, searchable=True, className='aggr')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['color_col' ,dcc.Dropdown(line_colors, id='aggr_color_col' , clearable = True, searchable=True, className='aggr')], style = {'width':'300px', 'float': 'left'})
], id='aggr_div', style={'display':'none'})

para_UI = html.Div([
  html.Div(['result_cats'     , dcc.Dropdown(result_cats                           , id='para_result_cats'   , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['lifecycle'       , dcc.Dropdown(json_column_keys_dict['lifecycle']    , id='para_lifecycle'     , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['setup'           , dcc.Dropdown(json_column_keys_dict['setup']        , id='para_setup'         , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['setupFilters'    , dcc.Dropdown(json_column_keys_dict['setupFilters'] , id='para_setupFilters'  , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['options'         , dcc.Dropdown(json_column_keys_dict['options']      , id='para_options'       , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['entries'         , dcc.Dropdown(json_column_keys_dict['entries']      , id='para_entries'       , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'}),
  html.Div(['exits'           , dcc.Dropdown(json_column_keys_dict['exits']        , id='para_exits'         , multi = True, clearable = True, searchable=True, className='para')], style = {'width':'300px', 'float': 'left'})
], id='para_div', style={'display':'none'})


app.layout = html.Div([  #html.Div([  dcc.Upload(html.Button('Upload File', style={'aligned':'center'}), id='upload_file')  ], id='upl_Container'),
html.Div([
  dcc.Store(id='graph_sel', data = json.dumps('sctr')),
  dcc.Store(id='sui_state', data = json.dumps([axis_options[0], axis_options[1], None, None, None, None, None])),
  dcc.Store(id='sui3d_state', data = json.dumps([axis_options[0], axis_options[1], axis_options[2], None, None, None, None])),
  dcc.Store(id='aggr_state', data = json.dumps([line_x_axis[0], line_y_axis[1], None])),
  dcc.Store(id='para_state', data = json.dumps(result_cats)),
  dcc.Store(id='options_state'),
  dcc.Store(id='sdf'),#, data=df_full.to_json(date_format='iso', orient='split')),
  dcc.Store(id='upload_status', data=json.dumps('Failure')),
  dcc.Store(id='upload_path', data=json.dumps('/N/A/')),
  dcc.Store(id='mem_df', data=df_full.to_json(date_format='iso', orient='split')),
  

  html.Div([
    html.H2("Data Selector", style = {'height':'5%'}),
    du.Upload(id='uploader', max_file_size=100, filetypes=['csv']  ),#  , upload_id='1234'),
    html.Div(id='H1Aw' ),# , style={'display':'none'}),
    html.Div(id='H2Aw', style={'display':'none'}),
    #dcc.Upload(html.Button('Upload File', style={'aligned':'center'}), id='upload_file'),
    #html.Hr(),

    html.Div([
      html.H6("Date Range of Data", style={'float':'top'}),
      dcc.DatePickerRange(
        id='dateRange',
        #display_format='MMM Do, YYYY',
        #min_date_allowed=st_date,
        #max_date_allowed=en_date,
        initial_visible_month=st_date,
        start_date=st_date,
        end_date=en_date,
        style={'margin':'0.5%', 'zoom':'auto', 'width':'99%', 'height':'33%', 'background-color': '#b0c4de'}
      )
    ], style = {'height':'14%', 'borderStyle': 'solid', 'borderColor': '#0F00', 'margin':'0.4%'}),
    html.Hr(),

    html.Div([
      html.H6("Data Options", style = {'height':'40%'}),
      html.Div([
        daq.BooleanSwitch(id='LiveDat', on=True, label='Live Data', style = {'float':'left', 'margin-right':'10px'}),
        daq.BooleanSwitch(id='bckTest', on=True, label='Backtest Data', style = {'float':'left', 'margin-right':'10px'}),
        daq.BooleanSwitch(id='fltrDat', on=True, label='Filter Data?', style = {'float':'left', 'margin-right':'10px'}),
      ], style = {'height':'60%'}), 
    ], style = {'height':'14%', 'borderStyle': 'solid', 'borderColor': '#0F00', 'margin':'0.4%'}),
    #html.H6('`'),
    html.Hr(style={'float':'center'}),

    
    html.Div(id='H3Aw', style={'display':'none'}),
    html.Div(id='H4Aw', style={'display':'none'}),
    html.Div(id='H5Aw', style={'display':'none'}),
    html.Div(id='H6Aw', style={'display':'none'}),
    
    #html.Div([
    #  html.H3(".", id='H3Asw'),
    #  html.Div(id='H3Aw', style={'display':'none'}),
    #  html.H3("Data Filtering", style = {'display':'inline','margin-top':'40%'}),
    #  daq.BooleanSwitch(id='fltrDat', on=True, style={'display':'inline'}),
    #  
    #], style = {'height':'15%'}),
    #html.Button("Update Data", id='upt_butt', style = {'width':'90%'}),#, style = {'margin-left':'5%','margin-right':'5%', 'width':'90%'}),
    html.Div([
      html.H6("Plot Categories"),
      html.Button("Scatter Plot", n_clicks =0, id='bt_sctr', className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      html.Button("3D Scatter Plot", n_clicks =0, id='bt_st3d',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      html.Button("Aggregate Plot", n_clicks =0, id='bt_aggr',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      html.Button("Parallel Plot", n_clicks =0, id='bt_para',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      html.Button("Treemap Plot", n_clicks =0, id='bt_tmap',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      html.Button("Generic Plots", n_clicks =0, id='bt_gnrc',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      #html.Button("Plot_7", n_clicks =0, id='button_7',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      #html.Button("Plot_8", n_clicks =0, id='button_8',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'}),
      #html.Button("Plot_9", n_clicks =0, id='button_9',  className='plot_buttons', style = {'margin':'auto', 'width':'45%'})
    ] , id='plot_buttons', style = {'height':'54%', 'borderStyle': 'solid', 'borderColor': '#0F00', 'margin':'0.4%'})
  ],id = 'options_pane', style=style_left_pane),

  html.Div([
    html.Div([dcc.Graph(figure=fig, style={'height':'100%', 'width':'100%'}, id='graph')], id='Graph_pane', style=style_graph),
    html.Hr(),

    
    html.Div([ html.H6("Plot Option Selectors"),
      scatter_UI, scatter3d_UI, aggr_UI, para_UI
    ], id='plot_options_pane', style = style_plot_options)
  ], id='graph_pane', style=style_right_pane)
#], id='Main_Container', style={'display':'none'})])# style=style_main)])
], id='Main_Container', style=style_main)])# style=style_main)])


# Make Callbacks

In [19]:
@du.callback(
    output=Output('sdf', 'data'),
    id="uploader",
)
def callback_on_completion(filenames):
    return json.dumps(filenames)

#for du >0.7
#@du.callback(
#    output=Output('sdf', 'data'),
#    id="uploader",
#)
#def callback_on_completion(status: du.UploadStatus):
#    return json.dumps(status.latest_file)

@app.callback(
    Output('upload_status', 'data'),
    Output('upload_path', 'data'),
    Output('H4Aw', 'children'),
    Output('dateRange', 'start_date'),
    Output('dateRange', 'end_date'),
    Output('para_lifecycle', 'options'),
    Output('para_setup', 'options'),
    Output('para_setupFilters', 'options'),
    Output('para_options', 'options'),
    Output('para_entries', 'options'),
    Output('para_exits', 'options'),
    [Input('sdf', 'data')]
)
def check_upload( df_path ):#, json_DF ):
  try:
    df__path = json.loads(df_path)
    red = pd.read_csv(df__path)
    red, _, date_range, jckd,_ = preprocess_df(red)
    df__path = df__path.replace('.csv', '_preprocced.csv')
    red.to_csv(df__path, sep=',')
    return json.dumps('Success!'), json.dumps(df__path), 'success! '+str(json.loads(df_path)), red['E_Dates'].iloc[0], red['E_Dates'].iloc[-1], jckd['lifecycle'], jckd['setup'], jckd['setupFilters'], jckd['options'], jckd['entries'], jckd['exits']
  except Exception as e1:
    try:
      df__path = json.loads(df_path)[0]
      red = pd.read_csv(df__path)
      red, _, date_range, jckd,_ = preprocess_df(red)
      df__path = df__path.replace('.csv', '_preprocced.csv')
      red.to_csv(df__path, sep=',')
      return json.dumps('Success!'), json.dumps(df__path), 'success! '+str(json.loads(df_path)), red['E_Dates'].iloc[0], red['E_Dates'].iloc[-1], jckd['lifecycle'], jckd['setup'], jckd['setupFilters'], jckd['options'], jckd['entries'], jckd['exits']
    except Exception as e:
      return json.dumps('Failure'), json.dumps(''), 'Failure! '+'E: '+str(e)+'F: '+str(df_path), df_full['E_Dates'].iloc[0], df_full['E_Dates'].iloc[-1], json_column_keys_dict['lifecycle'], json_column_keys_dict['setup'], json_column_keys_dict['setupFilters'], json_column_keys_dict['options'], json_column_keys_dict['entries'], json_column_keys_dict['exits']

"""
@app.callback(
    Output(component_id='mem_df', component_property='data'),
    Output('H5Aw', 'children'),
    [Input('upload_status', 'data'),
    Input('upload_path', 'data')]
)
def make_df( status, path ):
  status, path = json.loads(status), json.loads(path)
  if status=='Success!':
    try:
      red = pd.read_csv(path)
      red, _, _, _, _ = preprocess_df(red)
      return red.to_json(date_format='iso', orient='split'), 'Success!'+'F: '+str(path)
    except Exception as e:
      return df_full.to_json(date_format='iso', orient='split'), 'Failure! '+'F: '+str(path)+'E: '+str(e)
  else:
    return df_full.to_json(date_format='iso', orient='split'), 'Default'
"""


"""
@app.callback(#[Output('output-data-upload', 'children'),
              Output('sdf', 'data'),
              Input('upload_file', 'contents'),
              State('upload_file', 'filename'),
              State('upload_file', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        #return children, 
        return [parse_contents_df(c, n, d) for c, n, d in zip(list_of_contents, list_of_names, list_of_dates)][0].to_json(date_format='iso', orient='split')
"""

@app.callback(
    Output('graph_sel', 'data'),
    Output(component_id='sui_div', component_property='style'),
    Output(component_id='3dui_div', component_property='style'),
    Output(component_id='aggr_div', component_property='style'),
    Output(component_id='para_div', component_property='style'),
    Output(component_id='plot_options_pane', component_property='style'),
    Output(component_id='graph', component_property='style'),
    [Input('bt_sctr', 'n_clicks'),
    Input('bt_st3d', 'n_clicks'),
    Input('bt_aggr', 'n_clicks'),
    Input('bt_para', 'n_clicks'),
    Input('bt_tmap', 'n_clicks'),
    Input('bt_gnrc', 'n_clicks')]
)
def select_graph( btn1, btn2, btn3, btn4, btn5, btn6):
  changed_id = [p['prop_id'] for p in callback_context.triggered][0]
  if 'bt_sctr' in changed_id:
    cur_graph = 'sctr'
    return json.dumps(cur_graph), {'display':'block'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'block'}, {'height': '100%'}
  elif 'bt_st3d' in changed_id:
    cur_graph = 'sctr3d'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'block'}, {'display':'none'}, {'display':'none'}, {'display':'block'}, {'height': '100%'}
  elif 'bt_aggr' in changed_id:
    cur_graph = 'aggr'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'none'}, {'display':'block'}, {'display':'none'}, {'display':'block'}, {'height': '100%'}
  elif 'bt_para' in changed_id:
    cur_graph = 'para'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'block'}, {'display':'block'}, {'height': '100%'}
  elif 'bt_tmap' in changed_id:
    cur_graph = 'tmap'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'height': '150%'}
  elif 'bt_gnrc' in changed_id:
    cur_graph = 'gnrc'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'height': '150%'},
  else:
    cur_graph = 'None'
    return json.dumps(cur_graph), {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'display':'none'}, {'height': '100%'}


@app.callback(
    Output('sui_state', 'data'),
    [Input('sui_xaxis', 'value'),
    Input('sui_yaxis', 'value'),
    Input('sui_size_col', 'value'),
    Input('sui_color_col', 'value'),
    Input('sui_symbol_col', 'value'),
    Input('sui_facet_col', 'value'),
    Input('sui_hover_col', 'value')]
)
def sui_state_update( x,y,s,c,sym,f,h ):
  return json.dumps([x,y,s,c,sym,f,h])

@app.callback(
    Output('sui3d_state', 'data'),
    [Input('sui3d_xaxis', 'value'),
    Input('sui3d_yaxis', 'value'),
    Input('sui3d_zaxis', 'value'),
    Input('sui3d_size_col', 'value'),
    Input('sui3d_color_col', 'value'),
    Input('sui3d_symbol_col', 'value'),
    Input('sui3d_hover_col', 'value')]
)
def sui3d_state_update( x,y,z,s,c,sym,h ):
  return json.dumps([x,y,z,s,c,sym,h])

@app.callback(
    Output('aggr_state', 'data'),
    [Input('aggr_xaxis', 'value'),
    Input('aggr_yaxis', 'value'),
    Input('aggr_color_col', 'value')]
)
def aggr_state_update( x,y,c ):
  return json.dumps([x,y,c])

@app.callback(
    Output('para_state', 'data'),
    #Output('H3Aw', 'children'),
    [Input('para_result_cats', 'value'),
    Input('para_lifecycle', 'value'),
    Input('para_setup', 'value'),
    Input('para_setupFilters', 'value'),
    Input('para_options', 'value'),
    Input('para_entries', 'value'),
    Input('para_exits', 'value')]
)
def para_state_update(p1, p2, p3, p4, p5, p6 ,p7):
  #return json.dumps(list(p1)+list(p2)+list(p3)+list(p4)+list(p5)+list(p6)+list(p7))
  comb = []
  comb = comb+p1 if p1 else comb
  comb = comb+p2 if p2 else comb
  comb = comb+p3 if p3 else comb
  comb = comb+p4 if p4 else comb
  comb = comb+p5 if p5 else comb
  comb = comb+p6 if p6 else comb
  comb = comb+p7 if p7 else comb
  comb = comb if comb else result_cats
  return json.dumps(comb)

@app.callback(
    Output('options_state', 'data'),
    [Input('dateRange', 'start_date'),
    Input('dateRange', 'end_date'),
    Input('LiveDat', 'on'),
    Input('bckTest', 'on'),
    Input('fltrDat', 'on')]
)
def aggr_state_update( s,e, l,b,f ):
  ret = [s, e]
  ret = ret+[1] if l else ret+[0]
  ret = ret+[1] if b else ret+[0]
  ret = ret+[1] if f else ret+[0]
  return json.dumps(ret)


@app.callback(
    Output(component_id='graph', component_property='figure'),
    Output('H3Aw', 'children'),
    Output('H1Aw', 'children'),
    [Input('graph_sel', 'data'),
    Input('sui_state', 'data'),
    Input('sui3d_state', 'data'),
    Input('aggr_state', 'data'),
    Input('para_state', 'data'),
    Input('options_state', 'data'),
    Input('sdf', 'data'),
    Input('upload_path', 'data'),
    Input('upload_status', 'data'),
    #Input(component_id='wdf', component_property='data'),
     ]
)
def make_graph( graph_sel, sui_state, sui3d_state, aggr_state, para_state, options_state, json_DF, upload_path, upl_status):#, json_DF ):
  out_string = 'Upload Status: '
  try:
    ot_opts = json.loads(options_state)
    upl_status = json.loads(upl_status)
    if upl_status=='Success!':
      out_string = out_string + 'Updated'
      wdf = make_wdf(pd.read_csv(json.loads(upload_path)), ot_opts)
      #wdf = pd.read_csv(json.loads(upload_path))
      upl_status = upl_status+'_loaded_'+json.loads(upload_path)
    else:
      out_string = out_string + 'Default/Pending'
      wdf = make_wdf(df_full, ot_opts)
      upl_status = upl_status+'_NL'
    #wdf = pd.read_json(json_DF, orient='split')
    
    graph_sel = json.loads(graph_sel)
    if graph_sel == 'sctr':
      options = json.loads(sui_state)
      fig = make_scatter(wdf, options[0], options[1], options[2], options[3], options[4], options[5], options[6])
      return fig, str(upl_status), out_string
    elif graph_sel == 'sctr3d':
      options = json.loads(sui3d_state)
      return make_3dscatter(wdf, options[0], options[1], options[2], options[3], options[4], options[5], options[6]), str(options), out_string
    elif graph_sel == 'aggr':
      options = json.loads(aggr_state)
      return make_line(wdf, options[0], options[1], options[2]), str(options), out_string
    elif graph_sel == 'para':
      options = json.loads(para_state)
      return make_parallel(wdf, options), str(options), out_string
    elif graph_sel == 'tmap':
      options = 'tmap'
      return make_treemap_anim(wdf), str(options), out_string
    elif graph_sel == 'gnrc':
      options = json.loads(sui_state)
      fig = go.Figure()
      fig.add_trace(go.Scatter(x=[5,1,3,9], y=[5,-1, 7, -4]))
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      return fig, 'Generic selected', out_string
    else:
      fig = go.Figure()
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      return fig, 'None selected', out_string
  except Exception as e:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[0, 12, 12, 0, 0], y=[0, 0, 10, 10, 0]))
    fig.add_annotation(x=6, y=6, text="Couldn't generate plot due to following error:", showarrow=False, yshift=10)
    fig.add_annotation(x=6, y=5, text="{}".format(e), showarrow=False, yshift=10)
    fig.update_layout(paper_bgcolor="LightSteelBlue")
    return fig, 'Erreur', out_string

# Run Dash

In [20]:
if __name__ == '__main__':
  app.run_server(mode='external', debug=True)

Dash app running on:


<IPython.core.display.Javascript object>